In [17]:

import pandas as pd
import numpy as np
import pyreadr
from function_file import clean_names

In [18]:
# 1.1 Task 1 - Decathlon Data
# This data is contained in the .rds file decathlon.rds. You’ll need to use read_rds() from readr to open it.

# Read RDS Files
raw_rds = pyreadr.read_r('data_in\decathlon.rds') # also works for RData
# Convert RDS Files into dataframe
raw_df = raw_rds[None]

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
C:\Users\K\AppData\Local\Temp\ipykernel_11588\2218772229.py:5: SyntaxWarning: invalid escape sequence '\d'
  raw_rds = pyreadr.read_r('data_in\decathlon.rds') # also works for RData


In [19]:
# 1.1.1 Some cleaning hints
# This data has row names - how do you get rid of them?
# This data isn’t tidy.
# You will want better column header names.

clean_df = raw_df.copy()
#Convert Row Names into a column "Names"
clean_df = raw_df.reset_index().rename(columns={"rownames":"names"})
#Apply Clean Name function to clean column headers
clean_df = clean_df.rename(columns = clean_names)
clean_df["names"] = clean_df["names"].str.capitalize()

clean_df.columns

Index(['names', '100m', 'longjump', 'shotput', 'highjump', '400m',
       '110mhurdle', 'discus', 'polevault', 'javeline', '1500m', 'rank',
       'points', 'competition'],
      dtype='object')

In [20]:
#Return True if there is any Null values across columns within the dataframe
print(clean_df.isnull().any())

#Find any Null values across columns within the dataframe
print(clean_df.isnull().sum())

names          False
100m           False
longjump       False
shotput        False
highjump       False
400m           False
110mhurdle     False
discus         False
polevault      False
javeline       False
1500m          False
rank           False
points         False
competition    False
dtype: bool
names          0
100m           0
longjump       0
shotput        0
highjump       0
400m           0
110mhurdle     0
discus         0
polevault      0
javeline       0
1500m          0
rank           0
points         0
competition    0
dtype: int64


In [21]:
# 1.1.2 Analysis questions
# What was the average 100m time in each competition?
# Who had the highest total points across both competitions?
# What was the shot-put scores for the top three competitors in each competition?
# What was the average points for competitors who ran the 400m in less than 50 seconds vs. those than ran 400m in more than 50 seconds?

In [22]:
# Who had the longest long jump seen in the data?
max_longjump = clean_df[["names","longjump"]].loc[clean_df["longjump"].idxmax()]
max_longjump

names       Clay
longjump    7.96
Name: 14, dtype: object

In [23]:
# What was the average 100m time in each competition?
avg_100m = clean_df[["100m"]].mean()
avg_100m

100m    10.998049
dtype: float64

In [57]:
# Who had the highest total points across both competitions?
highest_pts = clean_df.copy()
# Remove the last 3 descriptive columns, prepare for pivot 
highest_pts = highest_pts.iloc[:,:-3]
#Pivot_longer
highest_pts = pd.melt(highest_pts, id_vars = ["names"])
# highest_pts["value"] = highest_pts.value.astype(float)
highest_pts = highest_pts.groupby(["names"], sort = True)["value"].sum()
highest_pts.sort_values(ascending = False).head(n=1)

names
Clay    1020.72
Name: value, dtype: float64

In [58]:
# Who had the highest total points across both competitions?
highest_pts_from_points = clean_df.copy()
highest_pts_from_points.loc[highest_pts_from_points["points"].idxmax()]

names            Sebrle
100m              10.85
longjump           7.84
shotput           16.36
highjump           2.12
400m              48.36
110mhurdle        14.05
discus            48.72
polevault           5.0
javeline          70.52
1500m            280.01
rank                  1
points             8893
competition    OlympicG
Name: 13, dtype: object

In [78]:
# What was the shot-put scores for the top three competitors in each competition?
short_put_per_compet = clean_df.copy()
short_put_per_compet = short_put_per_compet[["names","shotput","competition"]].sort_values(["competition", "shotput"], ascending=[True, False])

short_put_per_compet.groupby(["competition"]).head(n=3)

C:\Users\K\AppData\Local\Temp\ipykernel_11588\3225422068.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  short_put_per_compet.groupby(["competition"]).head(n=3)


,names,shotput,competition
4,Yurkov,15.19,Decastar
0,Sebrle,14.83,Decastar
2,Karpov,14.77,Decastar
13,Sebrle,16.36,OlympicG
15,Karpov,15.93,OlympicG
16,Macey,15.73,OlympicG


In [80]:
# What was the average points 
# for competitors who ran the 400m in less than 50 seconds vs. 
# those than ran 400m in more than 50 seconds?

avg_pts_400m = clean_df.copy()
#Create a new boolean column for 400m >= 50s or 400m < 50s
#True = Faster or Equal to 50 seconds
#False = 50 Seconds or Less
avg_pts_400m["faster_than_50"] = avg_pts_400m["400m"]>=50

avg_pts_400m.groupby("faster_than_50")["points"].mean()

faster_than_50
False    8120.482759
True     7727.166667
Name: points, dtype: float64